# **Tensorflow-Keras and Scikit-Learn With MLRun**

_______________________________________________________________________________

MLRun is an open-source Python package that provides a framework for running machine learning tasks transparently in multiple, scalable, runtime environments.  MLRun provides tracking of code, metadata, inputs, outputs and the results of machine learning pipelines. 

In this notebook we"ll compose a pipeline that deploys a classifier model, and uses it as the input in a training and validation step. We'll be working with a synthetic features matrix of dimension 10 million rows by 20 features and a binary label.  The model will be a 2-layer neural net classifier using **[tensorflow-keras](https://www.tensorflow.org/)** (v2.0.0b1), without gpu support.

The dataset we create is balanced, however there is a `weight` parameter in the data generator function specifying the fraction of observations that are labeled 0/False. The number of samples and features are also parameters.  The demonstration could be modified easily to allow for a more fine-grained control over the simulated dataset either by adding more parameters or replacing the underlying function altogether.

The training and validation step employs a scikit learn `Pipeline` to perform feature engineering. Some of the feature engineering needs to be done _**after**_ the train-valid-test set split. In some preprocessing scenarios we might estimate a data transformation on the training set before model training, and then apply the estimate to the validation and test sets before prediction. Since we need to perform the same transformation pre-inference, all pipeline model steps are stored.

Serializing models can be challenging for number of reasons:  a pipeline with multiple steps may require just as many encoding and decoding routines--applying joblib to a Keras model that has been wrapped in a scikit-learn api fails.  Since we have the model architecture in a class definition, all we need to do is save the weights.  Some steps in a pipeline may have no internal state to store, while others can be stored and loaded using `joblib`.  Most of it all boils down to storing dicts/json with numpy objects.

One of the upsides of the present architecture is that we can mix many simulations of data with a given model estimator, or many models with a given data sample and track everything in **MLRun**.  Research, development, and deployment, all on one page, running under multiple configurations, limited only by the compute resources at our disposal.


#### **notebook take-aways**
* write and test reusable and replaceable **[MLRun](https://github.com/mlrun)** components in a notebook, file or github repository
* store and load models
* run the components as a **[KubeFlow](https://www.kubeflow.org/)** pipeline

<a id='top'></a>
#### **steps**
**[install the python mlrun package](#install)**<br>
**[nuclio code section](#nuclio-code-section)**<br>
    - [nuclio's ignore](#ignore)<br>
    - [function dependencies](#function-dependencies)<br>

**[components](#components)**<br>
    - [supporting functions](#imports)<br>
    - [data simulation](#data_generator)<br>
    - [feature engineering](#feateng)<br>
    - [a classifier](#classifier)<br>
    - [save and load pipeline model](#save-load)<br>
    - [training and validation](#train)<br>
**[local tests](#local-testing)**<br>
**[remote tests](#remote)**<br>
**[compose pipeline](#compose)**<br>
**[run](#run)**<br>

<a id="install" ></a>
______________________________________________

# **notebook installs**

The following will reinstall the latest development version of ```mlrun```:

    !pip uninstall mlrun -y

    !pip install -U git+https://github.com/mlrun/mlrun.git@development

Install the KubeFlow pipelines package ```kfp```. For more information see the **[KubeFlow documentation on nuclio](https://www.kubeflow.org/docs/components/misc/nuclio/)** and  **[Kubeflow pipelines and nuclio](https://github.com/kubeflow/pipelines/tree/master/components/nuclio)**. For logging the estimated machine learning models we'll use ```joblib```'s ```dump``` and ```load```. For more details see **[Joblib: running Python functions as pipeline jobs](https://joblib.readthedocs.io/en/latest/index.html)**.

    !pip install numpy==1.16.4
    !pip install -U kfp joblib 
    !pip install -U seaborn matplotlib 
    !pip install -U sklearn pandas tensorflow==2.0.0b1
    !pip install -U git+https://github.com/yjb-ds/functions-demo.git

<a id="nuclio-code-section"><a>
______________________________________________

# **nuclio code section**

<a id='ignore'></a>
### _nuclio's **ignore** notation_

You"ll write all the code that gets packaged for execution between the tags ```# nuclio: ignore```, meaning ignore all the code here and above, and ```# nuclio: end-code```, meaning ignore everything after this annotation.  The **[docs](https://github.com/nuclio/nuclio-jupyter#creating-and-debugging-functions-using-nuclio-magic)** also suggest another approach: we can use ```# nuclio: start``` at the first relevant code cell instead of marking all the cells above with ```# nuclio: ignore```.

See the **[nuclio-jupyter](https://github.com/nuclio/nuclio-jupyter)** repo for further information on these and many other **[nuclio magic commands](https://github.com/nuclio/nuclio-jupyter#creating-and-debugging-functions-using-nuclio-magic)** that make it easy to transform a Jupyter notebook environment into a platform for developing production-quality, machine learning systems.

The ```nuclio-jupyter``` package provides methods for automatically generating and deploying nuclio serverless functions from code, repositories or Jupyter notebooks. **_If you have never run nuclio functions in your notebooks, please uncomment and run the following_**:

    !pip install nuclio-jupyter

The following two lines _**should be in the same cell**_ and mark the start of your mchine learning coding section:

In [21]:
# nuclio: ignore
import nuclio 

<a id="function-dependencies"></a>
### _function dependencies_

The installs made in the section **[Setup](#Setup)** covered the Jupyter environment within which this notebook runs.  However, we need to ensure that all the dependencies our nuclio function relies upon (such as ```matplotlib```, ```sklearn```, ```lightgbm```), will be available when that code is wrapped up into a nuclio function _**on some presently unknown runtime**_.   Within the nuclio code section we can ensure these dependencies get built into the function with the ```%nuclio cmd``` magic command.

In [22]:
%nuclio cmd -c pip install git+https://github.com/yjb-ds/functions-demo.git
%nuclio cmd -c pip install matplotlib tensorflow==2.0.0b1 sklearn pandas numpy==1.16.4 joblib
%nuclio cmd -c pip install mlrun

We"ll use a standard base image here, however the build step can be shortened by preparing images with pre-installed packages.

In [23]:
%nuclio config spec.build.baseImage = "python:3.6-jessie"

%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'


<a id="support"></a>
### _imports_

Some of the functionality is provided in custom components within the ```functions``` package (found at the github repo **[function-demos](https://github.com/yjb-ds/functions-demo)**):<br>

- **[datasets](functions/datasets.py)**:&emsp;generate simulation data
- **[files](functions/file_fs.py)**:&emsp;&emsp;&emsp;save and load _remote_ files
- **[models](function/model_fs.py)**:&nbsp; &emsp;save, load, and instantiate models
- **[plots](functions/plot_fs.py)**:&emsp;  &emsp; sundry plotting functions
- **[tables](functions/tables.py)**:&emsp; &nbsp; &nbsp;logging and retrieving table artifacts

In [24]:
from functions.datasets import create_binary_classification
from functions.tables import log_context_table, get_context_table
from functions.models import (FeaturesEngineer, 
                              Classifier, 
                              class_instance,
                              pipeline_save,
                              pipeline_load)

In [25]:
import os
from typing import Any, Union, Optional, List

In [26]:
import joblib
import json
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [27]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import pyarrow as pa

In [28]:
from sklearn.preprocessing import StandardScaler

In [29]:
from mlrun.execution import MLClientCtx
from mlrun.datastore import DataItem

<a id='paths'></a>
### _paths and parameters_

In [30]:
TARGET_PATH = '/User/mlrun/simdata'

# data simulation and ml training parameter
BATCH_SIZE = 1_024
LEARNING_RATE = 0.1
EPOCHS= 3
N_SAMPLES = 100_000
M_FEATURES = 20
CLASS_BALANCE = 0.5
DROPOUT = 0.5

<a id="components" ></a>
______________________________________________

# **components**

<a id='data_generator'></a>
## **data generator**

In [31]:
def data_generator(
    context: MLClientCtx,
    samples: int,
    features: int,
    features_hdr: Optional[List[str]],
    neg_weight: float,
    target_path: str,
    key: str
) -> None:
    """Generate raw data for this pipeline
    
    This component will be the entry point of the pipeline.
    
    In this demonstration our component is a simple wrapper for scikit learn's 
    `make_classification`, a convenient utility enabling us to build
    and test a pipeline from start to finish with a clean and 
    predictable dataset. By fiddling with neg_weight, we can also take a 
    quick look at the effect of class balance on our model before exposing it
    to the kind of data we find in the real world.
    
    :param context:       function context
    :param samples:       number of samples (rows) to generate
    :param features:      number of features (cols)
    :param features_hdr:  (optional) header for the features array
    :param neg_weights:   fraction of negative samples
    :param target_path:   destination for data including file name
    :param key:           context key of data
    """
    if features_hdr:
        assert len(features_hdr)==m_features, f"features header dimension mismatch for {name}"
    data = create_binary_classification(
                context, n_samples=samples, m_features=features,
                features_hdr=features_hdr,  weight=neg_weight, 
                target_path=target_path, key=key)

<a id='feateng'></a>
## **feature engineering**

This class implements the scikit-learn transformer API, enabling it to fit into an sklearn `Pipeline` as a step.<br>

For code please see the custom sklearn transformer `FeaturesEngineer` in **[models.py](functions/models.py)**.  

<a id="classifier"></a>
## **classifier**

This method generates a small keras Sequential model with 2 layers which gets wrapped in a `KerasClassifier` class. The latter provides it with a convenient sklearn interface for use in **[sklearn Pipelines](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn-pipeline-pipeline)**. The list of metrics collected during training can also be found in the same module as `METRICS` and includes accuracy, precision, recall, auc and a confusion matrix.<br>

For code please see `KerasClassifier` and `classifier_gen` in **[models.py](functions/models.py)**.  

<a id='save-load'></a>
## **save and load pipeline models**

The model presented here has three stages, wrapped into an sklearn pipeline:
1. **feature engineer**&emsp;&emsp; `function.models.FeaturesEngineer`
2. **scaler**&nbsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp;&nbsp;&nbsp; `sklearn.preprocessing.StandardScaler`
3. **classifier**&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;`functions.models.Classifier`, a `KerasClassifier` wrapper

In order to save this model pipeline, each of its components may have to be saved independently.

In our pipeline, the `FeaturesEngineer` has no state so we just create a new one during load. 

The `StandardScaler`'s estimates need to be re-used when transforming new data, so it is pickled using `joblib` (it contains arrays) with `xxxxx-scaler.pickle` as a filename.

Keras models have an architecture saved as json, and corresponding weights are saved in hdf5 format. The architecture can be recreated by instantiating the class or converting the json representation into a model, with the weights loaded into that structure. Filenames for the model are `xxxxx-weights.h5` and `xxxxx-model.json`.

For code please see `pipeline_save` and `pipeline_load` in **[models.py](functions/models.py)**.  

<a id='train'></a>
## **training and validation**

In this notebook demonstration we follow standard practice by wrapping the training/validation and test steps into the same method.

**exercise / todos**

To complete the demonstration, instead of hard-coding the `train_test_split` method, add a splitter class into the pipeline, like a cross-validator. 

The model encoder/decoder could also be input as a parameter.

In [32]:
def train(
    context: MLClientCtx,
    dataset: DataItem,
    engineer_cls: str,
    scaler_cls: str,
    classifier_cls: str,
    target_path: str,
    key: str = '',
    test_size: float = 0.1,
    valid_size: float = 0.3,
    batch_size: int = 1024,
    epochs: int = 5,
    verbose: bool = True,
    random_state: int = 1,
    ) -> None:
    """Train, validate, test and save a classifier model pipeline.
    
    Here we split the data, instantiate our pipeline and its models, and proceed
    to training and validation.
    
    :param context:             function context
    :param dataset:             cleaned input dataset
    :param engineer_cls:        feature engineering class
    :param scaler_cls:          scaler class
    :param classifier_cls:      classifier class    
    :param target_path:         destination for artifacts
    :param key:                 model key in context
    :param test_size:           (0.1) test set size as fraction
    :param valid_size:          (0.3) validation set size as fraction
    :param batch_size:          (1024) network feed batch size
    :param epochs:              (5) training epochs
    :param verbose:             (default True) Show metrics for 
                                training/validation steps
        
    Also included for demonstration are a randomly selected sample
    of training parameters:
    :param learning_rate: Step size at each iteration, constant.
    """
    raw = get_context_table(dataset)

    train, test = train_test_split(raw, test_size=test_size)
    train, valid = train_test_split(train, test_size=valid_size)
    
    y_train = train.pop('labels')
    y_valid = valid.pop('labels')
    y_test = test.pop('labels')

    # instantiate features engineer, scaler and classifier
    Engineer = class_instance(engineer_cls)
    Scaler = class_instance(scaler_cls)
    classifier = class_instance(classifier_cls)

    pipe = make_pipeline(Engineer(),
                         Scaler(),
                         classifier)
    pipe.fit(train, y_train)

    y_pred = pipe.predict(test)                          
    acc = accuracy_score(y_test, y_pred)
    context.log_result("accuracy", float(acc))

    pipeline_save(context, pipe, target_path, key)

#### _end of nuclio function definition_

In [33]:
# nuclio: end-code

<a id="local-testing" ></a>
______________________________________________

# **testing your code locally**

The function can be run locally and debugged/tested before deployment:

In [34]:
from mlrun import (mlconf,
                   code_to_function,
                   new_function,
                   NewTask,
                   new_model_server,
                   mount_v3io)  

Set MLRun's DB path.  MLRun wil generate and store all of its tracking and metadata to the `MLRUN_DBATH` environment variable.  We have set a `TARGET_PATH` earlier in this notebook in the above section **[paths and parameters](#paths)**.

In [35]:
mlconf.dbpath = 'http://mlrun-api:8080'

In [36]:
workflow = new_function()


In [37]:
datagen_run = workflow.run(
    name='data generator',
    handler=data_generator,
    params={
        'samples': N_SAMPLES,
        'features': M_FEATURES,
        'neg_weight': CLASS_BALANCE, # this is a balanced dataset
        'target_path': TARGET_PATH,
        'key': 'simdata'})

writing /User/mlrun/simdata/simdata-1e05X20.parquet



uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...19f310,0,Jan 02 16:45:00,completed,data generator,host=jupyter-788f6ff784-m6fh2,,samples=100000features=20neg_weight=0.5target_path=/User/mlrun/simdatakey=simdata,,features


to track results use .show() or .logs() or in CLI: 
!mlrun get run 09a3d0f3f28844b89954dde9f919f310  , !mlrun logs 09a3d0f3f28844b89954dde9f919f310 
[mlrun] 2020-01-02 16:45:01,529 run executed, status=completed


In [38]:
train_run = workflow.run(
    name='train, validate and store model',
    handler=train,
    inputs={
        'dataset': datagen_run.output('features')},
    params={
        'scaler_cls':     'sklearn.preprocessing.data.StandardScaler',
        'engineer_cls':   'functions.models.FeaturesEngineer',
        'classifier_cls': 'functions.models.Classifier',
        'target_path':     TARGET_PATH,
        'key':             'model_dir',
        'batch_size':      BATCH_SIZE,
        'epochs':          10})

Train on 63000 samples
63000/63000 [==============================] - 3s 41us/sample - loss: 0.3092 - tp: 29434.0000 - fp: 6637.0000 - tn: 24777.0000 - fn: 2152.0000 - accuracy: 0.8605 - precision: 0.8160 - recall: 0.9319 - auc: 0.9457



uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...b79363,0,Jan 02 16:45:01,completed,"train, validate and store model",host=jupyter-788f6ff784-m6fh2,dataset,scaler_cls=sklearn.preprocessing.data.StandardScalerengineer_cls=functions.models.FeaturesEngineerclassifier_cls=functions.models.Classifiertarget_path=/User/mlrun/simdatakey=model_dirbatch_size=1024epochs=10,accuracy=0.9657,model_dir


to track results use .show() or .logs() or in CLI: 
!mlrun get run ea24d5aac43a44e18a5e69eb42b79363  , !mlrun logs ea24d5aac43a44e18a5e69eb42b79363 
[mlrun] 2020-01-02 16:45:05,791 run executed, status=completed


<a id="image"></a>
#### _Create a deployment image_

Once debugged you can create a reusable image, and then deploy it for testing. In the following line we are converting the code block between the ```#nuclio: ignore``` and ```#nuclio: end-code``` to be run as a KubeJob. _**It is important to ensure that this function has been `deploy`ed at least once, and that you have access to it.**_

In [39]:
tfkeras_job = code_to_function(name='tfkeras_named_pipe',
                               runtime="job").apply(mount_v3io())

# set this to True so that updates to our git package are reflected in the built image,
# but please note however that this may lengthen image build times:
tfkeras_job.spec.no_cache = True

In [40]:
tfkeras_job.deploy()

[mlrun] 2020-01-02 16:45:08,289 starting remote build, image: .mlrun/func-default-tfkeras-named-pipe-latest
INFO[0000] Resolved base name python:3.6-jessie to python:3.6-jessie 
INFO[0000] Resolved base name python:3.6-jessie to python:3.6-jessie 
INFO[0000] Downloading base image python:3.6-jessie     
INFO[0000] Error while retrieving image from cache: getting file info: stat /cache/sha256:0318d80cb241983eda20b905d77fa0bfb06e29e5aabf075c7941ea687f1c125a: no such file or directory 
INFO[0000] Downloading base image python:3.6-jessie     
INFO[0000] Built cross stage deps: map[]                
INFO[0000] Downloading base image python:3.6-jessie     
INFO[0001] Error while retrieving image from cache: getting file info: stat /cache/sha256:0318d80cb241983eda20b905d77fa0bfb06e29e5aabf075c7941ea687f1c125a: no such file or directory 
INFO[0001] Downloading base image python:3.6-jessie     
INFO[0001] Unpacking rootfs as cmd RUN pip install -U git+https://github.com/yjb-ds/functions-demo.gi

True

<a id="remote"></a>
# **test your code remotely**

In [44]:
task = NewTask()
task.with_params(samples=N_SAMPLES,
                 features=M_FEATURES,
                 neg_weight=CLASS_BALANCE, # this is a balanced dataset
                 target_path=TARGET_PATH,
                 key='simdata')
nrun = tfkeras_job.run(task, 
                       handler='data_generator', 
                       out_path=TARGET_PATH)

[mlrun] 2020-01-02 16:58:35,518 starting run data_generator uid=6d05acac7dfd471d96efa3ae37d50c44  -> http://mlrun-api:8080
2020-01-02 16:58:44.454779: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2020-01-02 16:58:44.460348: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2500000000 Hz
2020-01-02 16:58:44.461450: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x50c9f60 executing computations on platform Host. Devices:
2020-01-02 16:58:44.461469: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
writing /User/mlrun/simdata/simdata-1e05X20.parquet

[mlrun] 2020-01-02 16:58:45,169 run executed, status=completed
final state: succeeded


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...d50c44,0,Jan 02 16:58:44,completed,tfkeras-named-pipe,host=data-generator-tcczwkind=jobowner=admin,,features=20key=simdataneg_weight=0.5samples=100000target_path=/User/mlrun/simdata,,features


to track results use .show() or .logs() or in CLI: 
!mlrun get run 6d05acac7dfd471d96efa3ae37d50c44  , !mlrun logs 6d05acac7dfd471d96efa3ae37d50c44 
[mlrun] 2020-01-02 16:58:47,811 run executed, status=completed


In [48]:
task.with_input('dataset', nrun.outputs['features'])
task.with_params(scaler_cls='sklearn.preprocessing.data.StandardScaler',
                 engineer_cls='functions.models.FeaturesEngineer',
                 classifier_cls='functions.models.Classifier',
                 target_path=TARGET_PATH,
                 key='model_dir',
                 batch_size=BATCH_SIZE,
                 epochs=10)

nrun = tfkeras_job.run(task,
                       handler='train',
                       out_path=TARGET_PATH)

[mlrun] 2020-01-02 17:01:31,935 starting run train uid=a4483f77adc8474a8664b2e0aa9e8601  -> http://mlrun-api:8080
2020-01-02 17:01:40.749845: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2020-01-02 17:01:40.756534: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2500000000 Hz
2020-01-02 17:01:40.757420: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x4f3f5e0 executing computations on platform Host. Devices:
2020-01-02 17:01:40.757444: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
2020-01-02 17:01:42.344896: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1483] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you w

uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...9e8601,0,Jan 02 17:01:40,completed,tfkeras-named-pipe,host=train-wdrm8kind=jobowner=admin,dataset,batch_size=1024classifier_cls=functions.models.Classifierengineer_cls=functions.models.FeaturesEngineerepochs=10key=model_dirscaler_cls=sklearn.preprocessing.data.StandardScalertarget_path=/User/mlrun/simdata,accuracy=0.964,model_dir


to track results use .show() or .logs() or in CLI: 
!mlrun get run a4483f77adc8474a8664b2e0aa9e8601  , !mlrun logs a4483f77adc8474a8664b2e0aa9e8601 
[mlrun] 2020-01-02 17:01:47,278 run executed, status=completed


<a id="compose"></a>
# **Create a KubeFlow Pipeline from our functions**

Our pipeline will consist of two steps, ```data_generator``` and ```train```.

For complete details on KubeFlow Pipelines please refer to the following docs:
1. **[KubeFlow pipelines](https://www.kubeflow.org/docs/pipelines/)**.
2. **[kfp.dsl Python package](https://kubeflow-pipelines.readthedocs.io/en/latest/source/kfp.dsl.html#module-kfp.dsl)**.

Please note, the model server file name in the ```new_model_server``` function call below should identical in every respect to the name of the model server notebook (here, **[model_server.ipynb](#model-server.ipynb)**).

In [49]:
import kfp
from kfp import dsl

In [50]:
srvfn = new_model_server("tfkeras",  
                         model_class="MyKerasClassifier",   
                         filename="model_server.ipynb")
srvfn.apply(mount_v3io())

In [51]:
@dsl.pipeline(
    name="Sklearn and KubeFlow",
    description="Shows how to use mlrun/kfp."
)
def tfkeras_pipeline(
    neg_weight = [0.5, 0.1],
):

    datagen = tfkeras_job.as_step(
        name='data generator',
        handler='data_generator',
        out_path=TARGET_PATH, 
        params={        
            'samples':         N_SAMPLES,
            'features':        M_FEATURES,
            'neg_weight':      CLASS_BALANCE,
            'target_path':     TARGET_PATH,
            'key':            'features'},
        outputs=['features']).apply(mount_v3io())
    
    train = tfkeras_job.as_step(
        name='sklearn pipe train',
        handler='train',
        out_path=TARGET_PATH, 
        inputs={'dataset': datagen.outputs['features']},
        outputs=['model_dir'],
        params={
            'scaler_cls':     'sklearn.preprocessing.data.StandardScaler',
            'engineer_cls':   'functions.models.FeaturesEngineer',
            'classifier_cls': 'functions.models.Classifier',
            'target_path':     TARGET_PATH,
            'key':             'model_dir',
            'batch_size':      BATCH_SIZE,
            'epochs':          10}).apply(mount_v3io())

    # define a nuclio-serving function, generated from a notebook file
    srvfn.deploy_step(project="mlrun_demos", 
                      models={"tfkeras_joblib": train.outputs["model_dir"]})

<a id="compile the pipeline"></a>
### _compile the pipeline_

We can compile our KubeFlow pipeline and produce a yaml description of the pipeline worflow:

In [52]:
# os.makedirs(TARGET_PATH, exist_ok=True)
kfp.compiler.Compiler().compile(tfkeras_pipeline, TARGET_PATH+"/mlrunpipe.yaml")

/User/.pythonlibs/jupyter/lib/python3.6/site-packages/kfp/components/_data_passing.py:133: UserWarning: Missing type name was inferred as "JsonArray" based on the value "[0.5, 0.1]".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))


In [53]:
client = kfp.Client(namespace="default-tenant")

Finally, the following line will run the pipeline as a job::

In [54]:
arguments = {
    'neg_weight' : [0.5, 0.1]    
}

run_result = client.create_run_from_pipeline_func(
    tfkeras_pipeline, 
    arguments, 
    run_name="tfkeras",
    experiment_name="tfkeras")

In [ ]:
# !mlrun clean